In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [2]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
wiki.name

'wikipedia'

In [4]:
## Vector Embedding And Vector Store
from langchain.vectorstores import FAISS, Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
import os
load_dotenv()
## Langmith tracking
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

# LANGCHAIN_TRACING_V2=True
# LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
# LANGCHAIN_API_KEY=""
# LANGCHAIN_PROJECT="Agent_new"

llm = ChatOllama(model='llama3.1:8b', temperature=0)

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

ollama_emb = OllamaEmbeddings(model="llama3.1:8b")

faiss_db = FAISS.from_documents(documents, ollama_emb)
chroma_db = Chroma.from_documents(documents, ollama_emb)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
retriever=faiss_db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x114cc3110>)

In [6]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(
	retriever,
	"langsmith_search",
	"Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
)

In [7]:
retriever_tool.name

'langsmith_search'

In [8]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [9]:
tools=[wiki,arxiv,retriever_tool]

In [10]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/sanketughadmathe/.pyenv/versions/3.12.2/envs/GenAI/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x112ad85e0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorsto

In [16]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
# Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/react")

In [17]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [18]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [19]:
agent_executor.invoke({"input":"Tell me about ML"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'Machine Learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton UniverBased on the tool call response, here's an answer to your original question:

Machine Learning (ML) is a field of study that focuses on developing algorithms and statistical models that enable computers to learn from data and make predictions or decisions without being explicitly programmed. The lecture notes mentioned in the tool call response provide a comprehensive overview of optimization techniques for machine learning, which are essential for training ML models.

Some key concepts in ML include:

1. Supervised Learning: This involves training a model on labeled data to predict outcomes.
2. Unsupervised Learning: This involves training a model on unlabeled data to identify patterns or rel

{'input': 'Tell me about ML',
 'output': "Based on the tool call response, here's an answer to your original question:\n\nMachine Learning (ML) is a field of study that focuses on developing algorithms and statistical models that enable computers to learn from data and make predictions or decisions without being explicitly programmed. The lecture notes mentioned in the tool call response provide a comprehensive overview of optimization techniques for machine learning, which are essential for training ML models.\n\nSome key concepts in ML include:\n\n1. Supervised Learning: This involves training a model on labeled data to predict outcomes.\n2. Unsupervised Learning: This involves training a model on unlabeled data to identify patterns or relationships.\n3. Deep Learning: This is a type of ML that uses neural networks with multiple layers to learn complex representations of data.\n\nThese concepts, and others like them, are crucial for understanding the field of Machine Learning."}

In [ ]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages
tools_new=[wiki]
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)


## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools_new,verbose=True)
agent_executor

In [ ]:
agent_executor.invoke({"input":"Tell me about ML"})

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent
prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)

In [ ]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools_new, verbose=True)

In [ ]:
agent_executor

In [ ]:
agent_executor.invoke(
    {
        "input": "Tell me about ML",
        # Notice that chat_history is a string, since this prompt is aimed at LLMs, not chat models
        # "chat_history": "Human: Hi! My name is Bob\nAI: Hello Bob! Nice to meet you",
    }
)